CNFAIC Observation Map
Goal:
Make a map of recent (<7 days) observations.  

Ideas:
- Custom date range
- Icons based on dates
- Professional only mode

Steps:
- Load observations and relevant data
- Assign location to observation
- Map observation
    - Format tags and popups
    
To Do:
- Make popups nice
    - Add border, for slides only?
    - Add icon for slide
- Add wx with different markers
- Create website with legend around map
- Layer options and menu clean-up

Low Priority:
- Fix encoding in notebook
- Fix flag count. There will be issues if other tables or length of avalanch table has additional rows


In [1]:
#Load libraries
import base64
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import folium
from folium import IFrame
import folium.plugins as plugins
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime

In [2]:
#Observation dataframe
Obs = pd.DataFrame(columns = ('Location', 'Date', 'Observer','Lat','Lon'))
Obs

#read observation archive

obsArchive = pd.read_pickle('./obsArchive.pkl')

In [3]:

def getObs(url):
    #Open url and convert to soup
    html = urlopen(url).read()
    soup = BeautifulSoup(html)
    with open('./Observations/'+ url[36:-1], 'a') as file: file.write(str(soup))
    
    #Pull location
    location = str(soup.find('title'))
    #location = location.replace("[\\u2018\\u2019]", "'") #Replace curly single quote with straight
    location = location[7:location.find('|') - 1]
    
    #Pull observer info, first check for anonymous report
    if str(soup.select_one(
        'div[ class *= cnfaic_obs-table-browse-observations-byline]')) \
        == '<div class="cnfaic_obs-table-browse-observations-byline">Anonymous</div>':
        observer = 'Anonymous'
    else:
        observer = soup.select_one("span[class *= cnfaic_obs-table-browse-observations-byline]").text
        observer = observer[:-1]
        
    #Pull date
    date = soup.select('div[ class *= "top_meta"]')
    date = date[1].text
    formattedDate = formatTime(date)
    
    #Pull coordinates
    if len(soup.find_all("a", href=lambda href: href and "google" in href)) > 0:
        links = soup.find_all("a", href=lambda href: href and "google" in href)
        coords = str(links[0])
        coords = coords[coords.find('q='):coords.find("'>")]
        lat = float(coords[coords.find('=') + 1 : coords.find(',') - 1])
        lon = float(coords[coords.find(",") + 1 : coords.find('target') - 2])
    else:
        lat = float('NaN')
        lon = float('NaN')
        
    #Red Flags
    if soup.find_all('tr') != []:
        table = soup.find_all('tr')
        
    
        rows = soup.find_all('tr')
        flags = []
        if len(rows) == 4: #Check for avalanche table, if avy table is there length is 7
            for row in rows:
                cols = row.find_all('td')
                cols = [ele.text.strip() for ele in cols]
                flags.append([ele for ele in cols if ele]) 
        else: #if avy table is above red flag table then this will work, assuming there isn't a table below it
            rows = rows[-3:]
            for row in rows:
                cols = row.find_all('td')
                cols = [ele.text.strip() for ele in cols]
                flags.append([ele for ele in cols if ele]) 
                
        recentAvy = flags[0][1] == 'Yes'
        collapsing = flags[1][1] == 'Yes'
        cracking = flags[2][1] == 'Yes'
    else:
        recentAvy = False
        collapsing = False
        cracking = False
    
    flagCount = 0
    if recentAvy == True:
        flagCount += 1
    if collapsing == True:
        flagCount += 1
    if cracking == True:
        flagCount += 1
        
    
    #Create dataframe from data
    d = {'Date':[formattedDate],'Location':[location], 'Observer':[observer],'Lat':[lat], 
         'Lon': [lon], 'Recent Avy':[recentAvy], 'Collapsing' : [collapsing], 
         'Cracking' : [cracking], 'flagCount' : flagCount, 'url' : [url]}
    oneObs = pd.DataFrame(data = d)
    
    #Pull red flags

    return(oneObs)

In [4]:
def getUrls():
    url = 'https://www.cnfaic.org/view-observations/'
    html = urlopen(url).read()
    soup = BeautifulSoup(html)
    table = soup.find('table')
    links = table.find_all('a')
    urls = list()
    gallery = 'gallery'
    i = 0
    for link in range(len(links)):
        if gallery in str(links[link]):
            i = i
        else:
            url = str(links[link])
            url = url[ 9 : url.find('>') - 1]
            if (obsArchive['url'] == url).any():
                i = i
            else:
                urls.append(url)
                i += 1
    return(urls)

    

In [5]:
def getNewObs():
    newObs = pd.DataFrame()
    urls = getUrls()
    for i in range(len(urls)):  #Changed this
        newObs = newObs.append(getObs(urls[i]))
    
    newObs.reset_index(inplace = True, drop = True)
    newObs['url'] = urls
    return(newObs)    

In [6]:
def addNewObs(obsArchive):
    newObs = getNewObs()

    for i in range(len(newObs)):
        obsArchive = obsArchive.append(newObs.iloc[i])       
        #Save the soup to the observations folder
        #with open('./Observations/'+ url[36:-1], 'a') as file: file.write(str(soup))
            
    obsArchive = obsArchive.sort_values('Date', ascending = False)    
    obsArchive.reset_index(inplace = True, drop = True)

    obsArchive.to_pickle('./obsArchive.pkl')
    return(obsArchive)
        

In [7]:
def formatTime(dateString):

    i = 0
    calendar = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', 'Jul':'07',
               'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}
    month = calendar[dateString[0:3]]

    day = dateString[dateString.find(',')-2:dateString.find(',')]
    if day[0] == ' ':
        day = '0' + day[1]

    year = dateString[dateString.find(',')+2:dateString.find(',')+6]
    year
    if dateString[-1] == 'm':
        time = dateString[-7:]
        if time[0] == ' ':
            time = '0' + time[1:]
    else:
        time = dateString[-5:]
        if time[0] == ' ':
            time = '0' + time[1:]

    dateString = str(year) + '-' + str(month) + '-' + str(day) + ' ' + time
    try:
        datetime.datetime.strptime(dateString, '%Y-%m-%d %H:%M%p')
    except:
        date = datetime.datetime.strptime(dateString, '%Y-%m-%d %H:%M')
    else:
        date = datetime.datetime.strptime(dateString, '%Y-%m-%d %H:%M%p')
    return(date)


In [101]:
obsArchive = addNewObs(obsArchive)
obsArchive

,Date,Location,Observer,Lat,Lon,Recent Avy,Collapsing,Cracking,flagCount,url,ageGroup
0,2020-12-17 08:44:00,Orca,Schauer/ Pfeiffer/Wunnicke Forecaster,60.934343,-149.111090,False,False,False,0,https://www.cnfaic.org/observations/orca-2/,NaN
1,2020-12-17 04:01:00,Summit – Manitoba,Caleb Rauch,60.684329,-149.418150,False,False,False,0,https://www.cnfaic.org/observations/summit-man...,NaN
2,2020-12-16 08:05:00,Magnum,Johnston-Bloom / Roberts Forecaster,60.753490,-149.200954,False,False,False,0,https://www.cnfaic.org/observations/magnum-55/,NaN
3,2020-12-15 08:55:00,Thousand Dollar Run,Rauch/Wright Kayes,61.767370,-149.298544,False,False,False,0,https://www.cnfaic.org/observations/thousand-d...,week
4,2020-12-15 02:00:00,Sunburst,Dylan Lenzie,60.794430,-149.199667,False,False,False,0,https://www.cnfaic.org/observations/sunburst-237/,week
...,...,...,...,...,...,...,...,...,...,...,...
137,2020-11-12 08:09:00,Girdwood Valley,George Creighton,60.995496,-149.093237,False,True,False,1,https://www.cnfaic.org/observations/girdwood-v...,old
138,2020-11-11 02:00:00,Tincan,Andy Moderow,60.785344,-149.164476,True,True,False,2,https://www.cnfaic.org/observations/tincan-287/,old
139,2020-11-10 12:00:00,Hiland Front Range,Anonymous,61.216874,-149.467287,False,False,False,0,https://www.cnfaic.org/observations/hiland-fro...,old
140,2020-11-10 06:23:00,Hatch peak,b s,61.766294,-149.305496,False,False,False,0,https://www.cnfaic.org/observations/hatch-peak...,old


In [9]:
"""formattedDates = []
for i in range(len(obsArchive)):
    formattedDates.append(formatTime(obsArchive.iloc[i][0]))

obsArchive['Date'] = formattedDates
    """

"formattedDates = []\nfor i in range(len(obsArchive)):\n    formattedDates.append(formatTime(obsArchive.iloc[i][0]))\n\nobsArchive['Date'] = formattedDates\n    "

In [10]:

# Load USGS
url_base = 'http://server.arcgisonline.com/ArcGIS/rest/services/'
service = 'NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}'
tileset = url_base + service

# Create the map
m = folium.Map(location = [60.79443,-149.199667], zoom_start = 11, tiles = tileset,
               attr='USGS Style')


# Add markers to map
"""for i in range(len(obsArchive)):
    if str(obsArchive.iloc[i][3]) != 'nan':
        folium.Marker([obsArchive.iloc[i][3],obsArchive.iloc[i][4]], popup = 
                      obsArchive.iloc[i][1] + ' ' +
                      '<a href="%s" target="_blank">Link</a>' % obsArchive.iloc[i][8]).add_to(m)
    """

'for i in range(len(obsArchive)):\n    if str(obsArchive.iloc[i][3]) != \'nan\':\n        folium.Marker([obsArchive.iloc[i][3],obsArchive.iloc[i][4]], popup = \n                      obsArchive.iloc[i][1] + \' \' +\n                      \'<a href="%s" target="_blank">Link</a>\' % obsArchive.iloc[i][8]).add_to(m)\n    '

In [106]:
#Filter obs archive into 3 groups based on age of obs
today = datetime.datetime.today()
oneDay = datetime.timedelta(days = 2)
threeDays = datetime.timedelta(days = 4)
oneWeek = datetime.timedelta(days = 8)
i = 0


for i in range(len(obsArchive)):
    if obsArchive.iloc[i][0].replace(hour = 0, minute = 0, second = 0) + oneDay > today:
        obsArchive['ageGroup'][i] = 'yesterday'
    elif obsArchive.iloc[i][0].replace(hour = 0, minute = 0, second = 0) + threeDays > today:
        obsArchive['ageGroup'][i] = '3 day'
    elif obsArchive.iloc[i][0].replace(hour = 0, minute = 0, second = 0) + oneWeek > today:
        obsArchive['ageGroup'][i] = 'week'
    else:
        obsArchive['ageGroup'][i] = 'old'


obsArchive




,Date,Location,Observer,Lat,Lon,Recent Avy,Collapsing,Cracking,flagCount,url,ageGroup
0,2020-12-17 08:44:00,Orca,Schauer/ Pfeiffer/Wunnicke Forecaster,60.934343,-149.111090,False,False,False,0,https://www.cnfaic.org/observations/orca-2/,yesterday
1,2020-12-17 04:01:00,Summit – Manitoba,Caleb Rauch,60.684329,-149.418150,False,False,False,0,https://www.cnfaic.org/observations/summit-man...,yesterday
2,2020-12-16 08:05:00,Magnum,Johnston-Bloom / Roberts Forecaster,60.753490,-149.200954,False,False,False,0,https://www.cnfaic.org/observations/magnum-55/,3 day
3,2020-12-15 08:55:00,Thousand Dollar Run,Rauch/Wright Kayes,61.767370,-149.298544,False,False,False,0,https://www.cnfaic.org/observations/thousand-d...,3 day
4,2020-12-15 02:00:00,Sunburst,Dylan Lenzie,60.794430,-149.199667,False,False,False,0,https://www.cnfaic.org/observations/sunburst-237/,3 day
...,...,...,...,...,...,...,...,...,...,...,...
137,2020-11-12 08:09:00,Girdwood Valley,George Creighton,60.995496,-149.093237,False,True,False,1,https://www.cnfaic.org/observations/girdwood-v...,old
138,2020-11-11 02:00:00,Tincan,Andy Moderow,60.785344,-149.164476,True,True,False,2,https://www.cnfaic.org/observations/tincan-287/,old
139,2020-11-10 12:00:00,Hiland Front Range,Anonymous,61.216874,-149.467287,False,False,False,0,https://www.cnfaic.org/observations/hiland-fro...,old
140,2020-11-10 06:23:00,Hatch peak,b s,61.766294,-149.305496,False,False,False,0,https://www.cnfaic.org/observations/hatch-peak...,old


In [12]:
dfYes = obsArchive[obsArchive['ageGroup'].isin(['yesterday'])]
df3day = obsArchive[obsArchive['ageGroup'].isin(['3 day'])]
dfWeek = obsArchive[obsArchive['ageGroup'].isin(['week'])]

In [99]:
# Load USGS
url_base = 'http://server.arcgisonline.com/ArcGIS/rest/services/'
service = 'NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}'
tileset = url_base + service

# Create the map
m = folium.Map(location = [60.79443,-149.199667], zoom_start = 11, tiles = tileset,
               attr='USGS Style')


# Add markers to map
"""for i in range(len(obsArchive)):
    if str(obsArchive.iloc[i][3]) != 'nan':
        folium.Marker([obsArchive.iloc[i][3],obsArchive.iloc[i][4]], popup = 
                      obsArchive.iloc[i][1] + ' ' +
                      '<a href="%s" target="_blank">Link</a>' % obsArchive.iloc[i][8]).add_to(m)
    """


fgObs = folium.FeatureGroup(name = 'Observations')
m.add_child(fgObs)
gYes = folium.plugins.FeatureGroupSubGroup(fgObs, 'Yesterday')
g3day = folium.plugins.FeatureGroupSubGroup(fgObs, '3 Days')
gWeek = folium.plugins.FeatureGroupSubGroup(fgObs, 'One Week')
subGroups = [gYes, g3day, gWeek]


frames = [dfYes, df3day, dfWeek]
colors = ['red', 'green', 'blue']
resolution, width, height = (72, 40, 40)

#iterate through the three time frames:
for i in range(len(frames)):
    m.add_child(subGroups[i])
    
    #iterate through the obs:
    for j in range(len(frames[i])):
        if str(frames[i].iloc[j][3]) != 'nan':  #only handle observations with coordinates
            filename = 'flag (' + str(int(frames[i].iloc[j][[8]])) + ').jpg' #load correct flag image
            encoded = base64.b64encode(open(filename, 'rb').read())
            image='<img src="data:image/jpeg;base64,{}">'.format 
            
            #create html for IFrame
            html = ('<b style="font-family:Helvetica,Arial,sans-serif; font-size: 16px;"> ' 
                    + '<a href="{}" target="_blank">{}</a>'.format(frames[i].iloc[j][9], frames[i].iloc[j][1])
                    + '</b> <br /> <body style="font-family:Helvetica,Arial,sans-serif; font-size: 14px;">'
                    + frames[i].iloc[j][2] + '<br />'
                    + frames[i].iloc[j][0].strftime("%m/%d/%Y, %H:%M %p") + '</body> <br>'
                    + image(encoded.decode('UTF-8')))
            #Create IFrame
            iframe = IFrame(html, width = 2000)
            popup = folium.Popup(iframe, min_width = 200, max_width = 2000)
            
            #add marker to subgroup
            subGroups[i].add_child(
                folium.Marker([frames[i].iloc[j][3],
                frames[i].iloc[j][4]], 
                popup = popup,
                icon = folium.Icon(color = colors[i])))

folium.LayerControl().add_to(m)
m


In [48]:
'<a href="{}" target="_blank">{}</a>'.format(frames[0].iloc[0][9], frames[0].iloc[0][1])

'<a href="https://www.cnfaic.org/observations/thousand-dollar-run-6/" target="_blank">Thousand Dollar Run</a>'

Working Zone

In [15]:
#Better Pop-ups
#Create new map to practice popups on:
# Load USGS
url_base = 'http://server.arcgisonline.com/ArcGIS/rest/services/'
service = 'NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}'
tileset = url_base + service

# Create the map
testMap = folium.Map(location = [60.79443,-149.199667], zoom_start = 11, tiles = tileset,
               attr='USGS Style')
i = 0
testFG = folium.FeatureGroup('popup test')
#Select images for the pop-up:
Filename ='flag (' + str(int(obsArchive.iloc[0][[8]])) + ').jpg'
encoded = base64.b64encode(open(Filename, 'rb').read())
image='<img src="data:image/jpeg;base64,{}">'.format 
#resolution, width, height = 72, 40, 40
"""iframe = IFrame(html(encoded.decode('UTF-8')), width=(width)+10, 
                height=(height)+10)
popup = folium.Popup(iframe, max_width=1000)
icon = folium.Icon(color="red", icon="ok")
marker = folium.Marker(location=[60.794430,	-149.199667], popup=popup, icon=icon)
marker.add_to(testMap)"""


testMap.add_child(folium.Marker([obsArchive.iloc[0][3],obsArchive.iloc[0][4]], 
                                popup = '<style> h1 {white-space: nowrap; font-size: 120%;}</style> <h1>' +
                                '<a href="%s" target="_blank">%s</a>' % (obsArchive.iloc[0][9], obsArchive.iloc[0][1]) +
                                '</h1>' +
                                obsArchive.iloc[0][2] + '<br>'+
                                obsArchive.iloc[0][0].strftime("%m/%d/%y %H:%M",) +
                                image(encoded.decode('UTF-8')) ,
                                icon = folium.Icon(color = colors[i])))

testMap

In [16]:
Filename ='flag (' + str(int(obsArchive.iloc[0][[8]])) + ').jpg'
encoded = base64.b64encode(open(Filename, 'rb').read())
html='<img src="data:image/jpeg;base64,{}">'.format 
resolution, width, height = 72, 40, 40
iframe = IFrame(html(encoded.decode('UTF-8')), width=(width)+10, 
                height=(height)+10)
popup = folium.Popup( iframe,min_width = 500, max_width=1000)
icon = folium.Icon(color="red", icon="ok")
marker = folium.Marker(location=[60.794430,	-149.199667], popup=popup, icon=icon)
marker.add_to(testMap)

In [17]:
encoded = base64.b64encode(open('flag (' + str(obsArchive.iloc[0][8]) + ').jpg','rb').read())

In [18]:
testObs = getObs('https://www.cnfaic.org/observations/eddies-111/')

In [19]:
testMap

In [20]:
url_base = 'http://server.arcgisonline.com/ArcGIS/rest/services/'
service = 'NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}'
tileset = url_base + service

# Create the map
testMap = folium.Map(location = [60.79443,-149.199667], zoom_start = 11, tiles = tileset,
               attr='USGS Style')

popup = folium.Popup(testObs.iloc[0][1])
marker = folium.Marker(location = [60.794430, -149.118785], popup = popup, icon = folium.Icon(color = 'red'))
marker.add_to(testMap)
testMap

In [21]:
getObs('https://www.cnfaic.org/observations/eddies-111/')

,Date,Location,Observer,Lat,Lon,Recent Avy,Collapsing,Cracking,flagCount,url
0,2020-12-12 08:31:00,Eddie’s,Kakiko Ramos-Leon,60.799749,-149.118785,True,False,False,1,https://www.cnfaic.org/observations/eddies-111/


In [107]:
url = 'https://www.cnfaic.org/observations/raggedtop-17/'
html = urlopen(url).read()
soup = BeautifulSoup(html)



In [108]:
soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html><head>
<title>Raggedtop | Chugach Avalanche Center </title>
<meta content="width=device-width" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" rel="stylesheet"/>
<link href="https://www.cnfaic.org/wp-content/themes/cnfaic/style.css?v=020120" media="screen" rel="stylesheet" type="text/css"/>
<link href="//www.avantlink.com" rel="dns-prefetch"/>
<link href="//s.w.org" rel="dns-prefetch"/>
<link href="https://www.cnfaic.org/wp-content/plugins/instagram-feed-pro/css/sbi-styles.min.css?ver=5.8" id="sbi_styles-css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://www.cnfaic.org/wp-content/plugins/the-events-calendar/common/src/resources/css/

In [22]:
#Saving stuff down here: